In [4]:
import tweepy
import pandas as pd
import datetime as datetime

auth = tweepy.OAuth2BearerHandler("AAAAAAAAAAAAAAAAAAAAAH6nYwEAAAAAK%2BezpbKj7NUWoAMW6SbFOfFruD4%3Dj134xtdV6I1kcGl4a1wnAaa7N2W0hQCA9avu7sDyvEEVWjIwmy")
api = tweepy.API(auth)

In [7]:
#Searching the archive for tweets mentioning the mavs

#mavsSearch = api.search_full_archive('dev', 'mavs', maxResults = 100)

In [8]:
#Viewing the first tweet

import json
status = mavsSearch[0]
json_str = json.dumps(status._json)
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

NameError: name 'mavsSearch' is not defined

In [9]:
#Creates the mineTweet function which extracts the information from a query and returns a dictionary with tweet data
#BUG: hashtags label also collects other random data from entities

def mineTweets(query, fromDate=datetime.datetime.now()):
        data = []
        fromDate = datetime.datetime.strptime(str(fromDate), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')

        searchResult = api.search_full_archive('dev', query, fromDate=fromDate, maxResults = 100)
        for item in searchResult:
                mined = {
                        'tweet_id':        item.id,
                        'name':            item.user.name,
                        'screen_name':     item.user.screen_name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.text,
                        'hashtags':        item.entities['hashtags'],
                        'location':        item.place,
                        'is_verified':     item.user.verified,
                        'created_at':      item.created_at
                }
                try:
                        mined['retweet_text'] = item.retweeted_status.full_text
                except:
                        mined['retweet_text'] = 'None'
                try:
                        mined['quote_text'] = item.quoted_status.full_text
                        mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                        mined['quote_text'] = 'None'
                        mined['quote_screen_name'] = 'None'
                
                data.append(mined)
        return pd.DataFrame(data)

In [ ]:
#Mining tweets on the mavs, commented out because twitter access was removed due to too many requests for the day.

# mavs_df = mineTweets('mavs', '202202100019')
# mavs_df = pd.DataFrame()

# time = datetime.datetime.now()
# time = datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')
# time = int(time)

# while time > 202202090019:
#     mavs_df = mavs_df.append(mineTweets('mavs', time))
    
#     dtTime = datetime.datetime.strptime(str(time), "%Y%m%d%H%M%S")
#     if mavs_df.tweet_id.duplicated().sum() > 0:
#         dtTime = dtTime - datetime.timedelta(minutes=5)
#     else:
#         dtTime = dtTime - datetime.timedelta(minutes=10)
#     time = int(dtTime.strftime('%Y%m%d%H%M'))

# mavs_df

NameError: name 'mineTweets' is not defined

In [16]:
#Trying SNScrape Since the twitter API rate limited the scraper

import snscrape.modules.twitter as sntwitter

def mineSntwitter(query, maxCount):
    tweet_list = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i>maxCount:
            break
        tweet_list.append([
                    tweet.date,
                    tweet.id,
                    tweet.user.username,
                    tweet.content,
                    tweet.user.verified,
                    tweet.retweetCount,
                    tweet.likeCount,
        ])
        tweet_df = pd.DataFrame(tweet_list, columns=['Datetime','tweet_id', 'username', 'text', 'verified', 'retweet_count', 'like_count'])

    return tweet_df

In [17]:
#Using SNScrape to scrape on tweets mentioning the mavs for the 2021-22 Season
#TODO: Scrape for a list of queries relating to the mavs/whatever team and then drop the duplicates

mavs_df = mineSntwitter('mavs since:2021-10-20 until:2022-02-09', 100)
mavs_df.text

0      Parlay (0.5u):\n\nBruins/Pens u5.5\nSens/Canes...
1      @TheRealAlexis I got close when Mavs traded Ba...
2      Nah 49 seconds in against the mavs is absolute...
3      @slingy24 @TheRealAlexis you won’t be alive in...
4                                @jederdiahbuck And mavs
                             ...                        
96     @AGranad0 I'm not questioning his experience i...
97     PARROT PICK:\n\nMavs/Pistons UNDER 212 4U!!\n\...
98              @EnricoAmabile @AndBand7 @TheRealTaddy93
99      @isaacappelt I’m trying to be like you my boy! 😂
100    @Traininmybrain @StorieASpicchi @storieincuffi...
Name: text, Length: 101, dtype: object